In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tqdm import tqdm
import re
import scipy
import tensorflow
from tensorflow.keras.layers import Input,Flatten, Embedding, Reshape, Multiply, Dropout, Dense, Concatenate, GlobalAveragePooling1D
from tensorflow.keras.layers import Layer, SpatialDropout1D, GlobalMaxPooling1D, Bidirectional, GRU, LSTM
from tensorflow.keras.layers import Dot, TimeDistributed, BatchNormalization, Add, Multiply
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
#import keras.backend as K
from sklearn.utils import shuffle
import seaborn as sns
import math
import shap

In [2]:
PATH = "../data/mind_small/"
news = pd.read_csv(PATH + "news.tsv",header=None, sep="\t")
behaviors = pd.read_csv(PATH + "behaviors.tsv", header=None, sep="\t")
news.columns = ["news_id", "category", "sub_category", "title", "abstract", "url", "title_entities", "abstract_entities"]
behaviors.columns = ["idx", "user_id", "time", "history", "impressions"]
behaviors = behaviors.drop_duplicates(["user_id", "history"]) 
behaviors.dropna(subset=["user_id", "history"], inplace=True)

In [3]:
sessions = {}
for idx, row in behaviors.iterrows():
    sessions[row["user_id"]] = row["history"].split(" ")

users = []
clicks = []
for k, v in sessions.items():
    for elem in v:
        users.append(k)
        clicks.append(elem)

tuples = list(zip(users, clicks))
interactions = pd.DataFrame(tuples, columns=["user", "news_id"])
interactions = interactions[:10000]

In [4]:
merged = interactions.merge(news, on=["news_id"])
merged.head()

,user,news_id,category,sub_category,title,abstract,url,title_entities,abstract_entities
0,U13740,N55189,tv,tvnews,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi...",https://assets.msn.com/labs/mind/AAIORni.html,[],"[{""Label"": ""Pat Sajak"", ""Type"": ""P"", ""Wikidata..."
1,U10045,N55189,tv,tvnews,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi...",https://assets.msn.com/labs/mind/AAIORni.html,[],"[{""Label"": ""Pat Sajak"", ""Type"": ""P"", ""Wikidata..."
2,U85394,N55189,tv,tvnews,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi...",https://assets.msn.com/labs/mind/AAIORni.html,[],"[{""Label"": ""Pat Sajak"", ""Type"": ""P"", ""Wikidata..."
3,U78244,N55189,tv,tvnews,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi...",https://assets.msn.com/labs/mind/AAIORni.html,[],"[{""Label"": ""Pat Sajak"", ""Type"": ""P"", ""Wikidata..."
4,U27024,N55189,tv,tvnews,"'Wheel Of Fortune' Guest Delivers Hilarious, O...","We'd like to solve the puzzle, Pat: Blair Davi...",https://assets.msn.com/labs/mind/AAIORni.html,[],"[{""Label"": ""Pat Sajak"", ""Type"": ""P"", ""Wikidata..."


# 1. Preprocessing

In [5]:
# remove users which have fewer than 5 interacations
print("Len before removal: ",len(merged))
_keys = merged["user"].value_counts()[merged["user"].value_counts() > 5].keys()
merged = merged[merged["user"].isin(_keys)]
print("Len after removal: ",len(merged))


user_enc = LabelEncoder()
article_enc = LabelEncoder()
merged["user_id"] = user_enc.fit_transform(merged["user"].values)
merged["article_id"] = article_enc.fit_transform(merged["news_id"].values)




Len before removal:  10000
Len after removal:  9867


In [6]:
import nltk
from nltk.corpus import stopwords
# Helper functions
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

def text_to_list(text):
    text = text.split(" ")
    return text

In [7]:
def clean_title(df):
    df["title_cleaned"] = df.title.apply(func = make_lower_case)
    df["title_cleaned"] = df.title_cleaned.apply(func = remove_stop_words)
    df["title_cleaned"] = df.title_cleaned.apply(func = remove_punctuation)
    return df
def hyphen_to_underline(category):
    """
    Convert hyphen to underline for the subcategories. So that Tfidf works correctly
    """
    return category.replace("-","_")
merged = clean_title(merged)
merged["subcategory_cleaned"] = merged["sub_category"].apply(func = hyphen_to_underline)

In [8]:
vectorizer = TfidfVectorizer(analyzer="word", tokenizer=str.split)
item_ids = merged["article_id"].unique().tolist()
tfidf_matrix = vectorizer.fit_transform(merged["subcategory_cleaned"])
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix

<9867x164 sparse matrix of type '<class 'numpy.float64'>'
	with 9867 stored elements in Compressed Sparse Row format>

In [9]:
item_ids = merged["article_id"].tolist()

def get_item_profile(item_id):
    """
    item_id: the news article id
    Return: an array of each n-gram in the item article. 
        with their n-gram id in tfidf_feature_names and weight.
    """
    idx = item_ids.index(item_id) # returns the index to the item id
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile
    
def get_item_profiles(ids):
    #print(ids)
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_user_profile(person_id):
    interactions = merged[merged["user_id"] == person_id]["article_id"].values # gets all articles
    user_item_profiles = get_item_profiles(interactions)
    user_item_profiles = np.sum(user_item_profiles, axis=0)
    user_profile_norm = sklearn.preprocessing.normalize(user_item_profiles)
    return user_item_profiles
    
#t = build_user_profile(1)

In [10]:
def calculate_user_profiles(unique_user_ids):
    user_profiles = {}
    for idx in tqdm(unique_user_ids):
        token_relevance = build_user_profile(idx).tolist()[0]
        zipped = zip(tfidf_feature_names, token_relevance)
        s = sorted(zipped, key=lambda x: -x[-1])[:6]
        user_profiles[idx] = s
    return user_profiles
        
user_profiles = calculate_user_profiles(merged["user_id"].unique())


100%|██████████| 231/231 [00:08<00:00, 27.93it/s]


In [11]:
subcategory_to_id = {name: idx+1 for idx, name in enumerate(tfidf_feature_names)}
id_to_subcategory = {idx: name for name, idx in subcategory_to_id.items()}
id_to_subcategory[0] = "Null"
subcategory_to_id["Null"] = 0

In [12]:
# add all id-category to the userprofile in df
profile_array = []
for index, row in tqdm(merged.iterrows()):
    
    user_idx = row["user_id"]
    profile = user_profiles[user_idx]
    temp = []
    for keyword_tuple in profile:
        temp.append(subcategory_to_id[keyword_tuple[0]])
    profile_array.append(temp)
merged["profile"] = profile_array

9867it [00:00, 11710.53it/s]


In [13]:
# add the id-category to the news articles
merged["subcategory_to_int"] = [subcategory_to_id[cat] for cat in merged["subcategory_cleaned"].values]

user_unique = merged.drop_duplicates("user_id")
userid_to_profile = user_unique[["user_id", "profile"]].set_index("user_id").to_dict()["profile"]

category_enc = LabelEncoder()
merged["main_category_int"] = category_enc.fit_transform(merged["category"].values)
article_id_to_category_int = merged[["article_id", "main_category_int"]].set_index("article_id").to_dict()
article_id_to_category_int = article_id_to_category_int["main_category_int"]

article_id_to_subcategory_int = merged[["article_id", "subcategory_to_int"]].set_index("article_id").to_dict()
article_id_to_subcategory_int = article_id_to_subcategory_int["subcategory_to_int"]

# 2. Train test split

In [36]:
def train_test_split(df, user_id, article_id, have_timestamp, timestamp):
    """
    params: 
        col_1: user_id
        col_2: article_id
    """
    df_test = df
    if have_timestamp: # if df have timestamp; take last interacted article into test set
        df_test = df_test.sort_values(timestamp).groupby(user_id).tail(1)
    else:
        df_test = df_test.sort_values(user_id).groupby(user_id).tail(1)
    df_train = df.drop(index=df_test.index)
    
    assert df_test.shape[0] + df_train.shape[0] == df.shape[0]
    
    return df_train, df_test
df_train_true, df_test_true = train_test_split(merged, "user_id", "article_id", False, 0)

In [37]:
all_article_ids = merged["article_id"].unique()

def negative_sampling(train_df, all_article_ids, user_id, article_id):
    """
    Negative sample training instance; for each positive instance, add 4 negative articles
    
    Return user_ids, news_ids, category_1, category_2, authors_onehotencoded, titles
    """
    
    users, articles, labels = [], [], []
    user_item_set = set(zip(train_df[user_id], 
                            train_df[article_id]))
    num_negatives = 4

    for (u, i) in tqdm(user_item_set):
        users.append(u)        
        articles.append(i)
        labels.append(1)
        for _ in range(num_negatives):
            negative_item = np.random.choice(all_article_ids)
            while (u, negative_item) in user_item_set:
                negative_item = np.random.choice(all_article_ids)
            users.append(u)
            articles.append(negative_item)
            labels.append(0)
    
    users, articles, labels = shuffle(users, articles, labels, random_state=0)

    return pd.DataFrame(list(zip(users, articles, labels)), columns=["user_id","article_id", "labels"])

df_train = negative_sampling(df_train_true, all_article_ids, "user_id", "article_id")

100%|██████████| 9500/9500 [00:00<00:00, 49617.70it/s]


In [38]:
# For each user; for each item the user has interacted with in the test set;
    # Sample 99 items the user has not interacted with in the past and add the one test item  
    
def negative_sample_testset(ordiginal_df, df_test, all_article_ids, user_id, article_id):
    test_user_item_set = set(zip(df_test[user_id], df_test[article_id]))
    user_interacted_items = ordiginal_df.groupby(user_id)[article_id].apply(list).to_dict()
    users = []
    res_arr = []
    
    userid_to_true_item = {} # keep track of the real items
    for (u,i) in tqdm(test_user_item_set):
        interacted_items = user_interacted_items[u]
        not_interacted_items = set(all_article_ids) - set(interacted_items)
        selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
        test_items = selected_not_interacted + [i]
        temp = []
        for j in range(len(test_items)):
            temp.append([u, test_items[j]])
        
        res_arr.append(temp)
        userid_to_true_item[u] = i
    X_test = np.array(res_arr)
    X_test = X_test.reshape(-1, X_test.shape[-1])
    df_test = pd.DataFrame(X_test, columns=["user_id","article_id"])
    return X_test, df_test, userid_to_true_item
X_test, df_test, userid_to_true_item = negative_sample_testset(merged, df_test_true, merged["article_id"].unique(), "user_id", "article_id")
    
    

100%|██████████| 231/231 [00:00<00:00, 986.72it/s]


# 3. Evaluation seutp

In [39]:
def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

In [40]:
def evaluate_one_rating(model, user_id, all_articles, true_item):
    ### Reshaping to make it on the right shape ###
    expanded_user_id = np.array([user_id]*100).reshape((100,1))
    all_articles = np.array(all_articles).reshape(-1,1)
    
    # predictions
    predictions = model.predict([expanded_user_id, all_articles])
    predicted_labels = np.squeeze(predictions)
    top_ten_items = [all_articles[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    hr = getHitRatio(top_ten_items, true_item)
    ndcg = getNDCG(top_ten_items, true_item)
    return hr, ndcg

def evalaute_model(model, df_test, userid_to_true_item):
    print("Evaluate model")
    hits = []
    ndcgs = []
    users = df_test["user_id"].unique()
    for user_id in tqdm(users):
        user_df = df_test[df_test["user_id"] == user_id] # get the 100 samples for this user
        true_item = userid_to_true_item[user_id] # get the actual true item in the test set
        all_articles = user_df["article_id"].values # get all possible articles
        
        ht, ndcg = evaluate_one_rating(model, user_id, all_articles, true_item)
        hits.append(ht)
        ndcgs.append(ndcg)
    return hits, ndcgs

# 4. Model

In [41]:
# Params
num_users = len(merged["user_id"].unique())
num_items = len(merged["article_id"].unique()) 
dims = 20

In [42]:
#@tf.autograph.experimental.do_not_convert
def get_model(num_users, num_items, dims, dense_layers=[128, 64, 32, 8]):
    user_input = Input(shape=(1,), name="user")
    item_input = Input(shape=(1,), name="item")
    
    user_emb = Embedding(output_dim=dims, input_dim=num_users, input_length=1, name="mf_user_emb")(user_input)
    item_emb = Embedding(output_dim=dims, input_dim=num_items, input_length=1, name="mf_item_emb")(item_input)
    
    user_vecs = Reshape([dims])(user_emb)
    item_vecs = Reshape([dims])(item_emb)
    
    y = Dot(1, normalize=False)([user_vecs, item_vecs])
    
    y = Dense(1, activation="sigmoid")(y)
    
    
    model = Model(inputs=[user_input, item_input], outputs=y)
    model.compile(
        optimizer=Adam(0.01),
        loss="mse",
        metrics=["accuracy"],
    )
    return model

model = get_model(num_users, num_items, dims)

In [43]:
###### Training ########
user_input = df_train.iloc[:, 0].values.reshape((-1,1))
item_input = df_train.iloc[:, 1].values.reshape((-1,1))
labels = df_train.iloc[:, 2].values.reshape((-1,1))
print(user_input.shape, item_input.shape, labels.shape )

(47500, 1) (47500, 1) (47500, 1)


In [44]:
all_user_ids = df_train["user_id"].unique()

#user_input = df_train.iloc[:, 0].values.reshape((-1,1))
#profile_input = df_train.iloc[:, 1:6].values
#item_input = df_train.iloc[:, 7].values.reshape((-1,1))
#labels = df_train.iloc[:, 8].values.reshape((-1,1))


train_loss = []
val_loss = []
train_acc = []
val_acc = []

hits_list = []
ndcg_list = []
best_hits = 0
best_ndcgs = 0

epochs=2
for epoch in range(epochs):
    hist = model.fit([user_input, item_input], labels, epochs=1, shuffle=True, verbose=1, validation_split=0.1)
    
    train_loss.append(hist.history["loss"])
    train_acc.append(hist.history["accuracy"])
    val_loss.append(hist.history["val_loss"])
    val_acc.append(hist.history["val_accuracy"])
    
    hits, ndcgs = evalaute_model( model, df_test, userid_to_true_item)
    hits_list.append(np.average(hits))
    ndcg_list.append(np.average(ndcgs))
    
    temp_hits = np.average(hits)
    temp_ndcgs = np.average(ndcgs)
    if (temp_hits > best_hits):
        best_hits = temp_hits
    if temp_ndcgs > best_ndcgs:
        best_ndcgs = temp_ndcgs
    

1336/1336 [==============================] - 2s 1ms/step - loss: 0.1717 - accuracy: 0.7990 - val_loss: 0.1563 - val_accuracy: 0.8004


  0%|          | 1/231 [00:00<00:25,  9.20it/s]

Evaluate model


100%|██████████| 231/231 [00:07<00:00, 32.20it/s]

  44/1336 [..............................] - ETA: 1s - loss: 0.1033 - accuracy: 0.8388

1336/1336 [==============================] - 1s 956us/step - loss: 0.0911 - accuracy: 0.8748 - val_loss: 0.1905 - val_accuracy: 0.7413


  0%|          | 0/231 [00:00<?, ?it/s]

Evaluate model


100%|██████████| 231/231 [00:07<00:00, 30.89it/s]


In [45]:
print("Hit @ 10: {:.2f} \nNdcg @ 10: {:.2f}".format(best_hits, best_ndcgs))

Hit @ 10: 0.24 
Ndcg @ 10: 0.15


# 5. user profile eval

In [46]:
def get_recommendations(user_id, df):
    
    ## Setup ###
    display_items = df[df["user_id"] == user_id]["article_id"].values.reshape(-1, 1)
    user_ids = np.tile(np.array(user_id), display_items.shape[0]).reshape(-1,1)
    
    ## Preds ###
    predictions = model.predict([user_ids, display_items])
    predicted_labels = np.squeeze(predictions)
    top_ten_items = [display_items[i][0] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    return top_ten_items

In [47]:
def get_article_subcategory(article_id, df):
    """
    Return the article's category
        type: int
    """
    return article_id_to_subcategory_int[article_id]
def get_category_hit_ratio(user_profile, top_ten_categories):
    for profile in user_profile:
        for category in top_ten_categories:
            if profile == category:
                return 1
    return 0
def get_ndcgs_category(user_profile, top_ten_categories):
    for i in range(len(top_ten_categories)):
        item = top_ten_categories[i]
        for profile in user_profile:
            if item == profile:
                return math.log(2) / math.log(i+2)
    return 0

In [50]:
def get_category_hits_ndcg(df):
    hits_ten,ndcgs_ten = [], []
    hits_five, ndcgs_five = [], []
    
    for user_id in tqdm(df["user_id"].unique()):
        top_ten_articles = get_recommendations(user_id, df)
        top_ten_subcategories = [get_article_subcategory(_id, df) for _id in top_ten_articles]
        user_profile = userid_to_profile[user_id]

        hit_ten = get_category_hit_ratio(user_profile, top_ten_subcategories)
        ndcg_ten = get_ndcgs_category(user_profile, top_ten_subcategories)
        
        hit_five = get_category_hit_ratio(user_profile, top_ten_subcategories[:5])
        ndcg_five = get_ndcgs_category(user_profile, top_ten_subcategories[:5])
        
        hits_ten.append(hit_ten)
        ndcgs_ten.append(ndcg_ten)
        
        hits_five.append(hit_five)
        ndcgs_five.append(ndcg_five)
        
        
    return np.average(hits_ten), np.average(ndcgs_ten), np.average(hits_five), np.average(ndcg_five)
        
        
category_hits_ten, category_ndcg_ten, category_hits_five, category_ndcg_five = get_category_hits_ndcg(df_test)

100%|██████████| 231/231 [00:08<00:00, 27.45it/s]


In [51]:
print(category_hits_ten)
print(category_ndcg_ten)
print(category_hits_five)
print(category_ndcg_five)

0.9090909090909091
0.5834249535370122
0.7316017316017316
0.5
